<a href="https://colab.research.google.com/github/jdmedinatobon/proyectoMachineLearning/blob/master/CrearArchivoDescriptores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este cuaderno se describe el procedimiento realizado para calcular los descriptores de las imágenes de resonancia magnética. Esto es necesario, ya que se cuentan con 30 imágenes de 64 x 64 pixeles que describen el cerebro. Cada grupo de 30 imágenes cuenta con un total de 122800 valores por muestra, motivo por el cual es necesario tomar estos datos y calcular un número menor de descriptores para el entrenamiento de los modelos. Estos descriptores se almacenan en un archivo de texto que se puede encontrar en el repositorio del proyecto en: https://github.com/jdmedinatobon/proyectoMachineLearning.

El entrenamiento de los modelos se realizó en un cuaderno por aparte que se puede encontrar junto a este.

Es importante mencionar que antes de calcular los descriptores se realizó un preprocesamiento sobre las imágenes utilizando easy fMRI. El repositorio se esta herramienta, de donde también se puede instalar, se puede encontrar aquí: https://easyfmri.github.io/. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile

datosZip = zipfile.ZipFile("/content/drive/My Drive/Datos.zip", 'r')
datosZip.extractall()
datosZip.close()

In [2]:
#En esta celda se incluyen los comandos necesarios para instalar las librerias requeridas.
#Esta solo se debe ejecutar cada vez que se inicia el entorno de ejecución.
!pip install mahotas
!pip install progressbar2

     |████████████████████████████████| 1.5MB 3.1MB/s 
  Created wheel for mahotas: filename=mahotas-1.4.9-cp36-cp36m-linux_x86_64.whl size=4264015 sha256=fcac9080c3007c64f674750b8cdbcc0f5f23449dae7e85f5edc2cef78d9f1965
  Stored in directory: /root/.cache/pip/wheels/02/e7/50/b67eee3b19c479cc5b41f10230bf9c1556be6eaaf095c6e997
Successfully built mahotas


In [0]:
#Aqui se importan las librerias necesarias para correr el codigo.
#Antes de correr esta celda se deben ejecutar la anterior para instalar las librerias que hacen falta.
import os
import numpy as np
import nibabel as nib
import csv
import mahotas as mh
import progressbar

%tensorflow_version 1.x
import tensorflow as tf
import time
import scipy.stats as st

Primero era necesario realizar la lectura de los datos requeridos, a partir del archivo original. Dicho archivo se encuentra en formato BIDS. Este formato es una manera de organizar los datos correspondientes a imágenes neurológicas. En el siguiente link se puede encontrar una descripción detallada del formato: https://bids.neuroimaging.io/.

Cada archivo de imágenes cuenta también con otro archivo de etiquetas. Este indica el intervalo de tiempo en el cual la persona estaba pensando en un objeto, lugar o persona. Esta información se utilizó para asignar las etiquetas correctas correspondientes a cada grupo de 30 imágenes. Así, la siguiente celda permite obtener un arreglo de NumPy que describe las imágenes de resonancia magnética y otro arreglo que indica la etiqueta con la siguiente regla:


*   objeto -> 0
*   lugar -> 1
*   cara -> 2



In [0]:
#Es la ruta del archivo raiz de los datos preprocesados.
pathArchivoDatosPreprocesados = "drive/My Drive"

#Estas 2 variables se utilizan para recorrer cada carpeta de los datos.
sub = [0, 0]
run = 1

#Es el numero de secciones de 2 segundos que se tienen en cuenta  
numImg = 5

#Es la seccion inicial desde la cual se toman los datos de imagenes.
offsetImg = 3

#Funcion que retorna el nombre del archivo que contiene las imagenes de resonancia magnetica.
#pSub: Indica la persona a la cual se le tomaron las imagenes.
#pRun: Indica el run para una persona. Cada run incluye 180 muestras de imagenes.
def darNombreArchivoImagenes(pSub, pRun):
  return pathArchivoDatosPreprocesados + "/ds001497-download/sub-"+str(pSub[0])+str(pSub[1])+"/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_bold.nii.gz"

#Funcion que retorna el nombre del archivo que contiene las etiquetas correspondientes a las imagenes de resonancia magnetica.
#pSub: Indica la persona a la cual se le tomaron las imagenes.
#pRun: Indica el run para una persona. Cada run incluye 180 muestras de imagenes.
def darNombreArchivoEtiquetas(pSub, pRun):
  return pathArchivoDatosPreprocesados + "/ds001497-download/sub-"+str(pSub[0])+str(pSub[1])+"/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_events.tsv"

def darNombreArchivoImagenesPreprocesadas(pSub, pRun):
  return pathArchivoDatosPreprocesados + "/ds001497-download/sub-"+str(pSub[0])+str(pSub[1])+"/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_analyze.feat/filtered_func_data.nii.gz"

#Funcion que obtiene el conjunto de 30 imagenes de 64x64 utilizados para el proyecto.
def obtenerImagenes(pImagenes):

  imagenes = np.zeros((64,64,30,15*numImg))

  for indice in range(0,15):

    for i in range(0,numImg):
      imagenes[:,:,:,indice*numImg + i] = pImagenes[:,:,:,indice*13 + i + offsetImg]

  return imagenes

#Funcion que obtiene las etiquetas y las convierte a su correspondiente numero entero de acuerdo con la siguiente regla:
#objeto -> 0
#lugar -> 1
#cara -> 2
def leerEtiquetas(pArchivoEtiquetas):
  etiquetas = []

  with open(pArchivoEtiquetas) as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    for row in reader:
      
      etiqueta = row['trial_type']

      if(etiqueta == 'object'):

        et = 0
      elif(etiqueta == 'place'):

        et = 1
      elif(etiqueta == 'face'):

        et = 2
      else:
        print("Error en el formato. Existe una clase distinta a object, place o face")

      for i in range(0,numImg):
        etiquetas.append(et)

  return etiquetas

#Funcion que genera una matriz de NumPy con los datos leidos de los archivos.
def importarDatos():
  datosPreprocesados = np.zeros((900*numImg,64,64,30))
  etiquetasPreprocesados = np.zeros(900*numImg)

  contador = 0
  p = 0

  sub[0] = 0

  with progressbar.ProgressBar(max_value = 900*numImg) as bar:
    for s1 in range(1,11):#(1,11)

      if(s1 == 10):
        sub[0] = 1
        sub[1] = 0
      else:
        sub[1] = s1

      for run in range(1,7):#(1,7)
        archivoImagenes = darNombreArchivoImagenes(sub, run)
        archivoEtiquetas = darNombreArchivoEtiquetas(sub, run)

        imagenes = nib.load(archivoImagenes).get_fdata()
        etiquetas = leerEtiquetas(archivoEtiquetas)

        muestras = obtenerImagenes(imagenes)

        for indice in range(0,15*numImg):
            datosPreprocesados[15*numImg*contador+indice,:,:,:] = muestras[:,:,:,indice]
            etiquetasPreprocesados[15*numImg*contador+indice] = etiquetas[indice]

            p+=1
            bar.update(p)

        contador+=1

  return datosPreprocesados, etiquetasPreprocesados.astype(int)

def importarDatosPreprocesados():
  datosPreprocesados = np.zeros((900*numImg - 15*numImg,64,64,30))
  etiquetasPreprocesados = np.zeros(900*numImg - 15*numImg)

  contador = 0
  p = 0

  sub[0] = 0

  with progressbar.ProgressBar(max_value = 900*numImg - 15*numImg) as bar:
    for s1 in range(1,11):#(1,11)

      if(s1 == 10):
        sub[0] = 1
        sub[1] = 0
      else:
        sub[1] = s1

      for run in range(1,7):#(1,7)

        if not (s1 == 8 and run == 1):
          
          archivoImagenes = darNombreArchivoImagenesPreprocesadas(sub, run)
          archivoEtiquetas = darNombreArchivoEtiquetas(sub, run)

          imagenes = nib.load(archivoImagenes).get_fdata()
          etiquetas = leerEtiquetas(archivoEtiquetas)

          muestras = obtenerImagenes(imagenes)

          for indice in range(0,15*numImg):

              datosPreprocesados[15*numImg*contador+indice,:,:,:] = muestras[:,:,:,indice]
              etiquetasPreprocesados[15*numImg*contador+indice] = etiquetas[indice]

              p+=1
              bar.update(p)

          contador+=1

  return datosPreprocesados, etiquetasPreprocesados.astype(int)

In [6]:
#En esta linea de codigo se importan los datos y se almacenan en las variables datosPreprocesados (correspondiente a las imagenes) y en etiquetasPreprocesadas (las etiquetas).
datosPreprocesados, etiquetasPreprocesados = importarDatosPreprocesados()

np.save("drive/My Drive/DatosProyecto/preprocessed/Datos_3_7", datosPreprocesados)
np.save("drive/My Drive/DatosProyecto/preprocessed/Etiquetas_3_7", etiquetasPreprocesados)

100% (4425 of 4425) |####################| Elapsed Time: 0:02:01 Time:  0:02:01


Con la lectura de los datos terminada, el siguiente paso es calcular los descriptores. Se escogieron 17 descriptores. Los primeros 4 (llamados de primer orden) se obtienen a partir de las imágenes, mientras que los segundos 13 (denominados de segundo orden) requieren primero la construcción de la matriz de concurrencia. Los descriptores de primer orden se obtuvieron con las librerías de NumPy y de scikit-learn y aquellos de segundo orden fueron calculados con la librería de mahotas. Además, para los de segundo orden también se ignoraron los valores de 0, ya que estos no contienen ninguna información, ya que son el fondo negro en las imágenes que no corresponde a ningún tejido cerebral. Finalmente, estos descriptores fueron almecenados en un archivo de texto para ser cargados desde el archivo de entrenamiento de modelos. 

La lista de descriptores es la siguiente:

Primer Orden: 

*   Media
*   Varianza
*   Kurtosis
*   Skewness

Segundo Orden: 

PONER ESTAS
*   Media
*   Varianza
*   Kurtosis
*   Skewness
*   Media
*   Varianza
*   Kurtosis
*   Skewness
*   Media
*   Varianza
*   Kurtosis
*   Skewness
*   Skewness




In [0]:
#Calcula los descriptores de primer orden. Estos se refieren a los que se sacan a partir del histograma, la media, la varianza, etc.
def calcularDescriptoresPrimerOrden(pImagenes):

  mean = np.mean(pImagenes, axis = None)
  var = np.var(pImagenes, axis = None)
  kur = st.kurtosis(pImagenes, axis = None)
  ske = st.skew(pImagenes, axis = None)

  descriptores = np.append([mean, var],[kur, ske])

  return descriptores

#Calcula los descriptores a partir de un grupo de imagenes de fMRI.
def calcularDescriptores(pImagenes):
  #Faltan los descriptores del histograma
  descPrimer = calcularDescriptoresPrimerOrden(pImagenes)
  descSegundo = mh.features.haralick(pImagenes, ignore_zeros=True, preserve_haralick_bug=False, compute_14th_feature=False, return_mean=True, return_mean_ptp=False, use_x_minus_y_variance=False, distance=1)

  descriptores = np.append(descPrimer,descSegundo)

  return descriptores

#Funcion que retorna el texto de la descripcion del archivo de descriptores.
def darTextoIntroduccion(pEscala):
  texto = "Este archivo incluye los descriptores calculados a partir de los datos preprocesados. \nLos datos brutos fueron obtenidos del \
dataset de imágenes de resonancia magnética funcional que se puede encontrar en https://openneuro.org/datasets/ds001497/versions/1.0.1.\n\
A estos datos se les realizó un preprocesamiento utilizando fmriprep (ESTO TOCA VER SI SI SIRVE AL FIN) y finalmente se calcularon los \
descriptores con una escala de grises de " + str(pEscala) + ".\nEstos descriptores son (EL NUMERO QUE SEA) e incluyen:\nEnergía, etc.... (AGREGAR LOS QUE FALTAN Y ORDENARLOS CORRECTAMENTE).\n\
Finalmente, cada muestra incluye a la clase a la cual pertenece. Existen 3 clases y se indican con un número de 0 a 2 que corresponden a:\n\
objeto -> 0\n\
lugar -> 1\n\
rostro -> 2\n\
media;varianza;kurtosis;skewness;....;clase"#AGREGAR LAS QUE FALTAN

  return texto

#Funcion que retorna un string con los descriptores y clases separados por ;.
#Esta cadena sera utilizada para generar el archivo de texto con los descriptores calculados y su etiqueta.
def darLinea(pDescriptores, pEtiqueta):
  separador = ";"
  muestra = []
  return separador.join(pDescriptores.astype(str)) + separador + str(pEtiqueta)


#Funcion que se utiliza para eliminar los valores negativos en los datos, ya que se requiere que estos sean positivos y enteros.
def eliminarValoresNegativos(pDatos):

  datosNegativosEliminados = pDatos
  datosNegativosEliminados[datosNegativosEliminados < 0] = 0

  return np.round(datosNegativosEliminados, decimals = 0)

#Funcion que genera el archivo de texto con los datos de los descriptores y su correspondiente etiqueta.
#Este archivo recibe el nombre de datosDescriptores.txt e incluye los descriptores y sus etiquetas separadas por ;
#y una descripcion del archivo.
def crearArchivoDescriptores(pDatos):
  archivo = open("datosDescriptoresPreprocesadosV1_3_7.txt", "w+")
  datos = eliminarValoresNegativos(pDatos)

  intro = darTextoIntroduccion(escala)
  archivo.write(intro)
  archivo.write("\n")

  start = time.time()

  #Esto deberia iterar hasta 1800, pero por ahora menos.
  for i in progressbar.progressbar(range(0,datos.shape[0])):#900*numImg)):
    imagenes = datos[i,:,:,:]
    imagenes = escala*(imagenes/imagenes.max())
    imagenes = imagenes.astype(int)

    descriptores = calcularDescriptores(imagenes)
    linea = darLinea(descriptores, etiquetasPreprocesados[i])
    archivo.write(linea)
    archivo.write("\n")
    
  print("Archivo generado exitosamente.")
  print("Tiempo: {} segundos".format(time.time()-start))

  archivo.close()


In [9]:
escala = 2**9
crearArchivoDescriptores(datosPreprocesados)

100% (4425 of 4425) |####################| Elapsed Time: 0:30:53 Time:  0:30:53


Archivo generado exitosamente.
Tiempo: 1853.4879775047302 segundos


In [0]:
test = eliminarValoresNegativos(datosPreprocesados)